In [1]:
from tracker import Tracker
from video import Video
from data import Data
from plotter import Plotter
from adapter import Adapter
from unit_converter import Unit_converter
import column_filter as cf
from datetime import datetime
from forces import Forces
import re

In [2]:
reference_distance = 0.3
mass = 0.5
tracker = Tracker([6, 8, 10])
video = Video("videos/video0.mkv")
if not video.is_opened():
    print("Video not found")
    exit()

positions = []
objects = ["shoulder", "elbow", "wrist"]

In [3]:
for frame in video.get_frames():
    keypoints = tracker.get_keypoints(frame)

    if len(keypoints) > 0:
        positions.append(keypoints)


0: 384x640 1 person, 70.0ms
Speed: 4.0ms preprocess, 70.0ms inference, 1574.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.0ms
Speed: 2.1ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.7ms
Speed: 3.0ms preprocess, 9.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.0ms
Speed: 2.3ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

In [4]:
adapter = Adapter(positions, objects, video.get_height())
data = Data(adapter.get_adapted_data(), objects)
df = data.get_data()
df[['rx_wrist', 'ry_wrist']]

In [6]:
unit_converter = Unit_converter(df['r_wrist'].iloc[0], reference_distance, video.get_fps(), 1)
df = unit_converter.convert_position(df, cf.position_columns(df.columns))
df = unit_converter.convert_velocity(df, cf.velocity_columns(df.columns))
df = unit_converter.convert_acceleration(df, cf.acceleration_columns(df.columns))
df = unit_converter.convert_angular_velocity(df, ['angular_velocity'])
df = unit_converter.convert_time(df, ['time'])
plotter = Plotter(df)
df[['rx_wrist', 'ry_wrist']]

,rx_wrist,ry_wrist
0,1.849553,0.766352
1,1.848640,0.765334
2,1.847727,0.764316
3,1.846814,0.763298
4,1.845901,0.762280
...,...,...
356,1.896455,0.712663
357,1.898983,0.711539
358,1.901510,0.710416
359,1.904038,0.709292


In [7]:
df = Forces(df, mass).get_data_with_forces()
df[['rx_wrist', 'ry_wrist', 'fx_g', 'fy_g']]

,rx_wrist,ry_wrist,fx_g,fy_g
0,1.849553,0.766352,0,-4.905
1,1.848640,0.765334,0,-4.905
2,1.847727,0.764316,0,-4.905
3,1.846814,0.763298,0,-4.905
4,1.845901,0.762280,0,-4.905
...,...,...,...,...
356,1.896455,0.712663,0,-4.905
357,1.898983,0.711539,0,-4.905
358,1.901510,0.710416,0,-4.905
359,1.904038,0.709292,0,-4.905


In [8]:
df.to_csv(f"csv/data_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv", index = False)

In [9]:
reversed_unit_converter = unit_converter.get_reversed_unit_converter()
df = reversed_unit_converter.convert_position(df, cf.position_columns(df.columns))
df = reversed_unit_converter.convert_force(df, cf.force_columns(df.columns))
df[['rx_wrist', 'ry_wrist', 'fx_g', 'fy_g']]

,rx_wrist,ry_wrist,fx_g,fy_g
0,878.066667,363.822222,0.0,-2.59254
1,877.633333,363.338889,0.0,-2.59254
2,877.200000,362.855556,0.0,-2.59254
3,876.766667,362.372222,0.0,-2.59254
4,876.333333,361.888889,0.0,-2.59254
...,...,...,...,...
356,900.333333,338.333333,0.0,-2.59254
357,901.533333,337.800000,0.0,-2.59254
358,902.733333,337.266667,0.0,-2.59254
359,903.933333,336.733333,0.0,-2.59254


In [10]:
for column in [column for column in df.columns if re.match("ry_", column)]:
    df[column] = video.get_height() - df[column]
for column in [column for column in df.columns if re.match("fy_", column)]:
    df[column] = -1 * df[column]
df[['rx_wrist', 'ry_wrist', 'fx_g', 'fy_g']]

,rx_wrist,ry_wrist,fx_g,fy_g
0,878.066667,716.177778,0.0,1082.59254
1,877.633333,716.661111,0.0,1082.59254
2,877.200000,717.144444,0.0,1082.59254
3,876.766667,717.627778,0.0,1082.59254
4,876.333333,718.111111,0.0,1082.59254
...,...,...,...,...
356,900.333333,741.666667,0.0,1082.59254
357,901.533333,742.200000,0.0,1082.59254
358,902.733333,742.733333,0.0,1082.59254
359,903.933333,743.266667,0.0,1082.59254


In [12]:
vectors = [
    (df['rx_wrist'].values, df['ry_wrist'].values, df['fx_g'].values, df['fy_g'].values, 'g', (0, 255, 0)),
    (df['rx_wrist'].values, df['ry_wrist'].values, df['fx_not_g'].values, df['fy_not_g'].values, 'not_g', (255, 0, 0)),
]

In [13]:
video.show_with_vectors(vectors)

In [ ]:
plotter.show_plot()

In [ ]:
video.close()